In [1]:
import pandas as pd
import numpy as np
import cassiopeia as cas
import seaborn as sns
from os.path import join, exists
from os import makedirs, getcwd
from pandarallel import pandarallel
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

pandarallel.initialize(nb_workers=32)

import pickle

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
af_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.tsv"
dp_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.DP.tsv"
ad_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.AD.tsv"
#donors = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex/cells_meta.tsv"
prefix= "TcellDupi_may17_2021/MTblacklist"
name = "post_cass_test_missingV"
outdir = "./output/data/"
af_thresh = 0.01
dp_thresh = 2

In [3]:
outdir = join(outdir, prefix)
if not exists(outdir):
    print(f"Making outdir {outdir} in folder {getcwd()}")
    makedirs(outdir)

In [4]:
af = pd.read_csv(af_f, index_col=0, sep='\t')
dp = pd.read_csv(dp_f, index_col=0, sep='\t').astype(int)
ad = pd.read_csv(ad_f, index_col=0, sep='\t').astype(int)


## Create character matrix - binarize af by af_thresh, and set any element under dp_thresh to -1

In [5]:
#Binarize
af = af.applymap(lambda x: 0 if x<af_thresh else 1)
af

,1040C,10634C,10646T,1082G,10912A,1095C,11123A,11137C,11175C,11204A,...,9531G,954G,9590T,9640T,9658G,971C,9750A,9823A,983T,9861G
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AAACGAAAGATTGACA-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AAACGAAAGCGTTGCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACGAAAGGTACTCT-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
AAACGAAAGGTTAACA-1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCCGGAAAG-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TTTGTGTTCCTGACCT-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TTTGTGTTCGGATGTT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Log fraction of 0s for af and dp

In [6]:
print((af == 0).sum().sum())

print(af.shape[0]*af.shape[1])

(af == 0).sum().sum()/(af.shape[0]*af.shape[1])

2403838
2693668


0.8924032211839024

In [7]:
print((dp == 0).sum().sum())

print(dp.shape[0]*dp.shape[1])

(dp == 0).sum().sum()/(dp.shape[0]*dp.shape[1])

2356952
2693668


0.8749972156925055

In [8]:
def dp_where(x, dp, dp_thresh):
#    print('name', x.name)
    curr = dp.loc[x.name]<dp_thresh
    x.loc[curr] = -1 #"-"
    return x
af = af.parallel_apply(dp_where, axis=1, args=(dp, dp_thresh))

## Few are just empty AFs, most dont have enough depth

In [9]:
print("Number of afs as 0",(af == 0).sum().sum()/(af.shape[0]*af.shape[1]))

print((af == -1).sum().sum())

print(af.shape[0]*af.shape[1])

(af == -1).sum().sum()/(af.shape[0]*af.shape[1])

Number of afs as 0 0.017405634250397598
2356953
2693668


0.8749975869335048

In [10]:
character_matrix = af.copy()
#character_matrix = af.parallel_apply(dp_where, axis=1, args=(dp,dp_thresh))
character_matrix

,1040C,10634C,10646T,1082G,10912A,1095C,11123A,11137C,11175C,11204A,...,9531G,954G,9590T,9640T,9658G,971C,9750A,9823A,983T,9861G
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGATTGACA-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGCGTTGCC-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGGTACTCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
AAACGAAAGGTTAACA-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCCGGAAAG-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
TTTGTGTTCCTGACCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
TTTGTGTTCGGATGTT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Each cell has at least one variant, and each variant seen in each cell (following 2 lines)

In [11]:
((character_matrix==0) | (character_matrix==-1)).all(axis=0).any()

False

In [12]:
((character_matrix==0) | (character_matrix==-1)).all(axis=1).any()

False

## Rename variants

In [13]:
var_map = {val:f"r{i}" for i, val in enumerate(character_matrix.columns) }
character_matrix = character_matrix.rename(var_map, axis=1)

In [14]:
character_matrix.to_csv(join(outdir, "missing.csv"))

## Reconstructing Trees

In [15]:
priors=None
if priors is not None:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix, priors=priors)
else:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix)
cas_tree.character_matrix.head(5)


,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r258,r259,r260,r261,r262,r263,r264,r265,r266,r267
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGATTGACA-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGCGTTGCC-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGGTACTCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
AAACGAAAGGTTAACA-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [16]:
cas_tree.n_cell, cas_tree.n_character,


(10051, 268)

In [17]:
# cell_meta = clone_allele_table.groupby('cellBC').agg({"intBC": 'nunique', 'UMI': 'sum', 'sampleID': 'unique'})
# cell_meta['sampleID'] = [x[0] for x in cell_meta['sampleID']]

# missing_proportion = (character_matrix == -1).sum(axis=0) / character_matrix.shape[0]
# uncut_proportion = (character_matrix == 0).sum(axis=0) / character_matrix.shape[0]
# n_unique_states = character_matrix.apply(lambda x: len(np.unique(x[(x != 0) & (x != -1)])), axis=0)

# character_meta = pd.DataFrame([missing_proportion, uncut_proportion, n_unique_states], index = ['missing_prop', 'uncut_prop', 'n_unique_states']).T

# cas_tree.cell_meta = cell_meta
# cas_tree.character_meta = character_meta

## HybridSolver 
1. Top-down greedy solver followed by
2. Bottom-solve of any tipe. 

In [18]:
# REINSTANTIATE the bottom and top solvers
vanilla_greedy = cas.solver.VanillaGreedySolver()
if priors is not None:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=10000, weighted=True, seed=1234)
else:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=10000, weighted=False, seed=1234)


In [ ]:
hybrid_solver = cas.solver.HybridSolver(top_solver=vanilla_greedy, bottom_solver=ilp_solver, cell_cutoff=40, threads=24)
hybrid_solver.solve(cas_tree, logfile='example_hybrid.log')


example_hybrid.log
None
subtree 0


[2021-08-18 18:03:20,855]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-18 18:03:20,858]    INFO [ILPSolver] Convergence time limit: 500
[2021-08-18 18:03:20,861]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-18 18:03:20,864]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-18 18:03:20,866]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-18 18:03:20,869]    INFO [ILPSolver] MIP gap: 0.01


subtree 0


[2021-08-18 18:03:20,912]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-18 18:03:20,915]    INFO [ILPSolver] Convergence time limit: 500
[2021-08-18 18:03:20,919]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-18 18:03:20,921]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-18 18:03:20,924]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-18 18:03:20,926]    INFO [ILPSolver] MIP gap: 0.01


subtree 0


...

[2021-08-19 06:27:11,702]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-19 06:27:11,705]    INFO [ILPSolver] Convergence time limit: 500
[2021-08-19 06:27:11,708]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-19 06:27:11,711]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-19 06:27:11,713]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-19 06:27:11,716]    INFO [ILPSolver] MIP gap: 0.01
[2021-08-19 06:27:11,842]    INFO [ILPSolver] (Process: 45e9381bb6dd238afc614a8ac2821784) Estimating a potential graph with a maximum layer size of 10000 and a maximum LCA distance of 16.
[2021-08-19 06:29:06,813]    INFO [ILPSolver] (Process: 45e9381bb6dd238afc614a8ac2821784) Maximum layer size exceeded, returning network.


subtree 0


[2021-08-19 06:29:06,917]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-19 06:29:06,920]    INFO [ILPSolver] Convergence time limit: 500
[2021-08-19 06:29:06,923]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-19 06:29:06,926]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-19 06:29:06,929]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-19 06:29:06,932]    INFO [ILPSolver] MIP gap: 0.01
[2021-08-19 06:29:07,033]    INFO [ILPSolver] (Process: 260b5809fd76969bcd753c735150ef7f) Estimating a potential graph with a maximum layer size of 10000 and a maximum LCA distance of 9.
[2021-08-19 06:31:01,988]    INFO [ILPSolver] (Process: 260b5809fd76969bcd753c735150ef7f) LCA distance 0 completed with a neighborhood size of 324.
[2021-08-19 06:45:47,631]    INFO [ILPSolver] (Process: 260b5809fd76969bcd753c735150ef7f) LCA distance 1 completed with a neighborhood size of 873.
[2021-08-19 06:53:20,388]    INFO [ILPSolver] (

subtree 0


[2021-08-19 07:24:54,844]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-19 07:24:54,847]    INFO [ILPSolver] Convergence time limit: 500
[2021-08-19 07:24:54,849]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-19 07:24:54,852]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-19 07:24:54,856]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-19 07:24:54,859]    INFO [ILPSolver] MIP gap: 0.01
[2021-08-19 07:24:54,972]    INFO [ILPSolver] (Process: 4119e968be4fc010889fd4d381d113df) Estimating a potential graph with a maximum layer size of 10000 and a maximum LCA distance of 11.


In [ ]:
character_matrix.duplicated().any()

In [ ]:
character_matrix.duplicated()

In [ ]:
!ls | grep 'example_hybrid'


In [ ]:
rndict = {}
_iter = 0
for n in cas_tree.nodes:
    if ',' in n:
        rndict[n] = f'node{_iter}'
        _iter += 1

cas_tree.relabel_nodes(rndict)

In [ ]:
pickle.dump(cas_tree, open(join(outdir, name+".full.missingV.castree.p"), 'wb'))

# Plot tree 

## Plot with networkx


In [ ]:
G = nx.DiGraph()
G.add_nodes_from(cas_tree.nodes)
G.add_edges_from(cas_tree.edges)
G

# write dot file to use with graphviz
# run "dot -Tpng test.dot >test.png"

f = plt.figure(figsize=(10,10), dpi=300)
write_dot(G,join(outdir, f'{name}_hybrid.dot'))


# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos = graphviz_layout(G, prog='dot', root=cas_tree.root)
nx.draw(G, pos, with_labels=False, arrows=True)

# color_map = []
# for node in G:
#     if node < 10:
#         color_map.append('blue')
#     else: 
#         color_map.append('green')
# nx.draw(G, pos, node_color=color_map, with_labels=False)


plt.savefig(join(outdir, f'{name}_hybrid.png'))


## Plot with itol (only with valid itol account)

In [ ]:
# plot tree
cas.pl.upload_and_export_itol(cas_tree, itol_config="~/.itolconfig_hidden",
                              tree_name = "example_hybrid_tree", export_filepath="example_hybrid_tree.png")


## Test a sample of the data

In [ ]:
priors=None
if priors is not None:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix.sample(100).iloc[:,:50], priors=priors)
else:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix.sample(100).iloc[:,:50])
# REINSTANTIATE the bottom and top solvers
vanilla_greedy = cas.solver.VanillaGreedySolver()
if priors is not None:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=500, weighted=True, seed=1234)
else:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=500, weighted=False, seed=1234)

hybrid_solver = cas.solver.HybridSolver(top_solver=vanilla_greedy, bottom_solver=ilp_solver, cell_cutoff=40, threads=10)
hybrid_solver.solve(cas_tree, logfile='example_hybrid.log')


In [ ]:
cas_tree.character_matrix.shape

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(cas_tree.nodes)
G.add_edges_from(cas_tree.edges)
G

# write dot file to use with graphviz
# run "dot -Tpng test.dot >test.png"

f = plt.figure(figsize=(10,10), dpi=300)
write_dot(G,join(outdir, f'{name}_hybrid.dot'))

# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos = graphviz_layout(G, prog='dot', root=cas_tree.root)
nx.draw(G, pos, with_labels=False, arrows=True)

plt.savefig(join(outdir, f'{name}_smallSample_hybrid.png'))


In [ ]:
print